# Tracker Per-LS Data Exploration Tools

This notebook is intended to present a series of per-LS data exploration tools. It utilizes the DIALS Python API to fetch the monitoring element histograms, as well as the OMS API to obtain metadata on the run including the trigger rate. These tools offer the option to display the integral of a specified reference run in addition to the run under evaluation. This notebook is intended to be run in SWAN. However, in the case you wish to run it in a local virtual environment, a `pyproject.toml` is included which specifies all of the basic dependencies which you can install using Poetry. Note that, in either case, you will have to do it in `lxplus`.

NOTE: In order to run the OMS API you will need to have a json file with the client ID (`API_CLIENT_ID`) and secret (`API_CLIENT_SECRET`). For more information on how to obtain these, you can take a look at these [slides](https://indico.cern.ch/event/997758/contributions/4191705/attachments/2173881/3670409/OMS%20CERN%20OpenID%20migration%20-%20update.pdf).

## Setup

In [ ]:
# Run this if you are in SWAN
# Package installation/updating
!pip3 install -e .. --no-dependencies
!pip3 install omsapi
!pip3 install cmsdials --upgrade

In [ ]:
# DIALS API
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)

# creds = Credentials.from_creds_file()
dials = Dials(creds)

Run your prefered authentication method for the OMS API

In [ ]:
# OMS API
# For more information on the OMS API, please visit https://gitlab.cern.ch/cmsoms/oms-api-client

In [ ]:
# krb
import omsapi

oms_fetch = omsapi.OMSAPI("https://cmsoms.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_krb()

In [ ]:
# oidc
import omsapi
import json

with open("./clientid.json", "r") as file:
    secrets = json.load(file)

oms_fetch = omsapi.OMSAPI("http://vocms0185.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_oidc(secrets["API_CLIENT_ID"], secrets["API_CLIENT_SECRET"], audience="cmsoms-int-0185")

del(secrets)
del(file)

In [ ]:
# DQMExplore
import dqmexplore
from dqmexplore.plotting import plot1DMEs, plot2DMEs
from dqmexplore.exploreutils import check_empty_ls
from dqmexplore.dataproc import generate_me_dict, integrate
from dqmexplore.statplotting import plotheatmaps1D, plotMEs1D_static
from dqmexplore.omsutils import makeDF, get_rate, plot_rate

# Plotly
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

# Other useful libraries for data exploration, manipulation and processing
import numpy as np
import pandas as pd

If you run the following, you will get a list of all the available monitoring elements (MEs).

In [ ]:
mes_df = pd.DataFrame([me_qry_rslt.__dict__ for me_qry_rslt in dials.mes.list()])
mes_df

## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

You can access the trigger rate* in the following way:

*HLT ZeroBias trigger rate

In [ ]:
runnb = 380360
omstrig_df = get_rate(oms_fetch, runnb, "ZeroBias")
fig = plot_rate(omstrig_df, norm=False)
plot(fig, filename=f"./plots/trigrate_{runnb}")

## 1D Monitoring Elements

We fetch data from DIALS as shown here. For more information on how to use the DIALS Python API, please refer to the [official repository](https://github.com/cms-DQM/dials-py). If you are unfamiliar with regex syntax, you can take a look at the following [cheat sheet](https://www.rexegg.com/regex-quickstart.html) for a quick overview.

In [ ]:
# Getting current run data
runnb = 380466
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting current run trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [ ]:
# Checking for empty LSs, empty here meaning that the number of entries in the historgram is less than the set threshold (default=10)
print("Empty/Near empty LSs:")
check_empty_ls(generate_me_dict(data1D))

In [ ]:
# Getting reference data
refrun = 380360

refdata1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = refrun,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Defining plot features
ax_labels = [
    dict(
        x="Charge (e)", 
        y="Count"
    )
] * 4

fig_title = f"Pixel Barrel Charge Normalized (Run {runnb}, Ref {refrun})"

# Plotting
fig = plot1DMEs(
    data1D, 
    fig_title=fig_title,
    ax_labels=ax_labels, 
    norm=True,
    # trigger_rates=trig_rate[:-1], 
    ref_df=refdata1D
)

# Export plot to html
plot(fig, filename=f"./plots/PixelBarrelCharge-run{runnb}-ref{refrun}-normalized.html")

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken.

In [ ]:
runnb = 381147
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

# Getting trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

ax_labels = [
    dict(x = "Charge(e)", y = "LS")
] * 4

fig_title = f"Pixel Barrel Charge Heatmaps Normalized by Trigger Rate (Run {runnb})"

fig = plotheatmaps1D(
    data1D,
    fig_title=fig_title,
    trigger_rates = trig_rate[:-1],
    ax_labels=ax_labels,
    show=False,
    norm=True
)

# Export plot to html
plot(fig, filename=f"./plots/PixelBarrelChargeHeatmap-run{runnb}-trignorm.html")

## 2D Monitoring Elements

2D monitoring elements are also available in DIALS and they can be accessed as shown below.

In [ ]:
runnb = 378981
me__regex = "PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_."
# me__regex = "(PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_(1|2|3))|(PixelPhase1/Tracks/clusterposition_zphi_ontrack)"
# me__regex = "PixelPhase1/Tracks/PXForward/clusterposition_xy_ontrack_PXDisk_+."

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [ ]:
# Defining plot features
ax_labels = [
    dict(x="SignedModuleCoord", y="SignedLadderCoord")
] * 4

fig_title = f"Pixel Barrel Digi Occupancy Normalized by Trigger Rate (Run {runnb})"

fig2D = plot2DMEs(
    data2D,
    fig_title=fig_title,
    ax_labels=ax_labels,
    trigger_rates=trig_rate
)

# plot(fig2D, filename=f"PixelBarrelDigiOccupancy-run{runnb}-trignorm.html"))
fig2D.write_html("plot.html", include_plotlyjs="cdn")

## Integrated MEs

Suppose you found that the first 10 lumisections of a run have some sort of issue and you want to get the same plot the GUI would give you, but without including those LSs. This next section offers some tools to do just that. Run the following block of code to generate a slider widget which you can use to interactively select the LSs you wish to integrate over.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
range_slider = widgets.IntRangeSlider(
    value=[1, len(trig_rate)],
    min=1,
    max=len(trig_rate),
    step=1,
    description='Lumisections:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

output = widgets.Output()
display(output)

# List to store selected ranges
# Temporary list to hold currently selected, but unaded ranged
selected_range = []
# List of added ranges
final_ranges = []
# Full list of lss generated from list of added ranges
selected_lss = []

def ranges_overlap(range1, range2):
    start1, end1 = range1
    start2, end2 = range2
    return start1 <= end2 and start2 <= end1

# Slider interactivity functions
def on_slider_change(change):
    selected_range[:] = tuple(change['new'])
    update_output()

def update_output():
    with output:
        clear_output()
        print(f"Selected ranges: {selected_range}")

# Attch change handler to slider
range_slider.observe(on_slider_change, names='value')

display(range_slider, output)

# Buttons def
def clear_ranges(_):
    final_ranges.clear()
    update_output()

def apply_ranges(_):
    for frange in final_ranges:
        if ranges_overlap(frange, tuple(selected_range)):
            print("Error: Overlap of ranges. Try again.")
            break
    else:
        final_ranges.append(tuple(selected_range[:]))
        print(f"Range selected: {final_ranges}")
        
def get_lss(_):
    selected_lss.clear()
    for lsrange in final_ranges:
        selected_lss.extend(range(lsrange[0], lsrange[1]+1))
        selected_lss.sort()
    with output:
        print(f"Lumisection selection: {selected_lss}")

clear_button = widgets.Button(description="Clear Ranges")
clear_button.on_click(clear_ranges)

apply_button = widgets.Button(description="Apply Ranges")
apply_button.on_click(apply_ranges)

get_lss_button = widgets.Button(description="Get LSs")
get_lss_button.on_click(get_lss)

display(clear_button, apply_button, get_lss_button)

In [ ]:
print(selected_lss)

In [ ]:
# ax_labels = [
#     dict(x = "Charge (e)", y = "Count")
# ] * 4

ax_labels = [
    dict(x="SignedModuleCoord", y="SignedLadderCoord")
] * 4

fig_title = f"Pixel Barrel Charge Integrated (Run {runnb}, Ref {refrun})"

fig = plotMEs1D_static(
    data2D,
    norm=True,
    fig_title=fig_title,
    ax_labels=ax_labels,
    ls_filter=selected_lss,
    trigger_rates=trig_rate,
#     ref_df=refdata1D
)
# Export plot to html
plot(fig, filename=f"PixelBarrelCharge-Integrated-run{runnb}-ref{refrun}.html")